[View in Colaboratory](https://colab.research.google.com/github/Joovvhan/ColabTest/blob/master/Transfer_Learning_Model_Comparison.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
